In [1]:
import re
import time
from random import randint
from time import sleep
from urllib.request import Request, urlopen
import requests
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import (ElementClickInterceptedException,
                                        ElementNotInteractableException,
                                        NoSuchElementException,
                                        NoSuchWindowException,
                                        WebDriverException)
from selenium.webdriver.common.by import By

In [2]:
pd.set_option('display.max_column',None)

In [1]:
driver = webdriver.Chrome()
# Get one page by URL
url = 'https://propzy.vn/mua/bat-dong-san/hcm?tags=hcm&type=mua' # (url of page 1)
req = Request(url, headers = {'User-Agent': 'Mozilla/5.0'})
driver.get(url) 
driver.title

NameError: name 'webdriver' is not defined

In [5]:
def get_detail_link(): # get 32 links of a page 
  res = []
  links = driver.find_elements(By.CSS_SELECTOR, 'div.listing-info h3 > a')
  links = [x.get_attribute('href') for x in links]
  res += links
  return res 

page_1 = get_detail_link()

In [6]:
#  functions for 1 details url 
# get item_code
def get_item_code():
  try: 
    item_code = driver.find_element(By.XPATH, ' //*[@id="__next"]/div[1]/div/div/div[2]/div[2]/p[1]/span').text
  except NoSuchElementException:
    # print('item_code is Na with url:',driver.current_url)
    item_code = 'Na'
  return item_code

# get item_name
def get_item_name():
  try: 
    item_name = driver.find_element(By.TAG_NAME, 'h1').text
  except NoSuchElementException:
    # print('item_name is Na with url:',driver.current_url)
    item_name = 'Na'
  return item_name 

# get details 
def get_details_property():
  try: 
    details = driver.find_elements(By.ID, 'chi-tiet')
    sleep(randint(1,4))
    details = [x.text for x in details] 
  except NoSuchElementException:
    # print('details is Na with url:',driver.current_url)
    details = 'Na'
  return details 

# get total_price 
def get_total_price(): 
  try: 
    total_price = driver.find_element(By.CSS_SELECTOR, 'div.styles__PriceDetailsContainer-sc-195g207-6.clVCQF > p').text
  except NoSuchElementException:
    # print('total_price is Na with url:',driver.current_url) 
    total_price = 'Na' 
  return total_price

# get unit price - đơn giá đất 
def get_unit_price():
  try: 
    unit_price = driver.find_element(By.CSS_SELECTOR, 'div.styles__PriceDetailsContainer-sc-195g207-6.clVCQF > div > div > span').text
  except NoSuchElementException:
    # print('unit_price is Na with url:',driver.current_url)
    unit_price = 'Na' 
    # res.append(unit_price)
  return unit_price

#  get gross_unit_price - đơn giá gộp 
def get_gross_unit_price(): 
  try: 
    gross_unit_price = driver.find_element(By.CSS_SELECTOR,'div.styles__PriceDetailsContainer-sc-195g207-6.clVCQF > div > p > span').text
    sleep(randint(1,4))
  except NoSuchElementException:
    # print('gross_unit_price is Na with url:',driver.current_url)
    gross_unit_price = 'Na' 
    #  res.append(gross_unit_price)
  return gross_unit_price

# get facilites - số phòng ngủ , số phòng tắm, diện tích, hướng 
def get_facilities():
  # get bedroom - phòng ngủ 
  bedroom = driver.find_element(By.CSS_SELECTOR, 'div.styles__DetailsContainer-sc-195g207-9.hncFox > div:nth-child(1) > div > span').text
  if bedroom == '--':
    bedroom = 'Na'
  else: 
    bedroom = int(bedroom)
  # get bathroom- phòng tắm 
  bathroom = driver.find_element(By.CSS_SELECTOR, 'div.styles__DetailsContainer-sc-195g207-9.hncFox > div:nth-child(2) > div > span').text
  if bathroom == '--':
    bathroom = 'Na'
  else: 
    bathroom = int(bathroom)
  # get area - diện tích 
  area = driver.find_element(By.CSS_SELECTOR, 'div.styles__DetailsContainer-sc-195g207-9.hncFox > div:nth-child(3) > div > span').text
  # get direction -  hướng
  direction = driver.find_element(By.CSS_SELECTOR, 'div.styles__DetailsContainer-sc-195g207-9.hncFox > div:nth-child(4) > div > span').text
  return bedroom, bathroom, area, direction

# get address
def get_address():
  try: 
    address = driver.find_element(By.CSS_SELECTOR,' div.styles__MainContentContainer-sc-195g207-1.jEjOAA > p.styles__PropertyAddress-sc-195g207-4.hBmlrn').text
  except NoSuchElementException:
    # print('address is Na with url:',driver.current_url)
    address = 'Na' 
  return address
# get payment monthly 
def get_payment(): # 
  try: 
    payment = driver.find_element(By.CSS_SELECTOR, 'p.styles__PropertyPayment-sc-195g207-5.iEDlCq > button > span').text
  except NoSuchElementException:
    # print('payment is Na with url:',driver.current_url)
    payment = 'Na'
  return payment 
# get utilities -  tiện ích 
def get_utility():
  try: 
    utility = driver.find_element(By.CSS_SELECTOR, '#tien-ich > ul ').text 
  except NoSuchElementException:
    # print('utility is Na with url:',driver.current_url)
    utility = 'Na'
  return utility

In [7]:
# get df
def get_all(urls):
  res_item_code = []
  res_item_name = []
  res_address = []
  res_bedroom = []
  res_bathroom = []
  res_area = []
  res_direction = []
  res_details_property = []
  res_total_price = []
  res_unit_price = []
  res_gross_unit_price = []
  res_payment = []
  res_utility = []
  for url in urls: 
    req = Request(url, headers = {'User-Agent': 'Mozilla/5.0'})
    driver.get(url) 
    sleep(randint(3,5))
    #  mã tin 
    item_code = get_item_code()
    res_item_code.append(item_code)
    # tiêu đề
    item_name  = get_item_name()
    res_item_name.append(item_name)
    # địa chỉ 
    address = get_address()
    res_address.append(address)
    # cơ sở vật chất 
    bedroom, bathroom, area, direction = get_facilities()
    res_bedroom.append(bedroom)
    res_bathroom.append(bathroom)
    res_area.append(area)
    res_direction.append(direction)

    # chi tiết bđs 
    details_property = get_details_property()
    res_details_property.append(details_property)
    # giá 
    total_price = get_total_price()
    res_total_price.append(total_price)
    # đơn giá đất 
    unit_price = get_unit_price()
    res_unit_price.append(unit_price)
    # đơn giá gộp 
    gross_unit_price = get_gross_unit_price()
    res_gross_unit_price.append(gross_unit_price)
    # thanh toán hàng tháng (nếu vay)
    payment = get_payment()
    res_payment.append(payment)
    # tiện ích 
    utility = get_utility()
    res_utility.append(utility)
    
  df = pd.DataFrame(list(zip(res_item_code , res_item_name, res_bedroom, res_bathroom,res_area, res_direction,  res_address, res_details_property, res_total_price,res_unit_price, res_gross_unit_price, res_payment, res_utility)),
  columns = ['item_code', 'item_name', 'bedroom', 'bathroom', 'area', 'direction', 'address','details_property', 'total_price', 'unit_price', 'gross_unit_price','payment', 'utility'])
  return df



In [8]:
# read page_19 ( 640 items )
df = pd.read_csv('page_19.csv')
df.head()

,Unnamed: 0,link,brief,id,price,price_per_unit,address,facilities_key,facilities_value
0,0,https://propzy.vn/mua/can-ho/hcm/quan-5/id387193,Bán Chung Cư Cũ Gò Công Quận 5 - Gần Chợ Kim Biên,id387193,750 triệu,"53,57 triệu/m²","Đ.Gò Công, P.13, Q.5, Hồ Chí Minh","['ic ic-bed', 'ic ic-bath', 'ic ic-acreage', '...","['1', '1', '14m²', 'Đông']"
1,1,https://propzy.vn/mua/can-ho/hcm/huyen-binh-ch...,Bán căn hộ nhà ở xã hội huyện Bình Chánh,id417224,780 triệu,"22,34 triệu/m²","Đ.Hương Lộ 80, X.Vĩnh Lộc A, H.Bình Chánh,","['ic ic-bed', 'ic ic-bath', 'ic ic-acreage']","['1', '1', '34.91m²']"
2,2,https://propzy.vn/mua/nha/hcm/quan-10/id391821,Bán nhà hẻm an ninh Hồ Thị Kỷ P1 Q10 - Nhà kiê...,id391821,900 triệu,"209,3 triệu/m²","Đ.Hồ Thị Kỷ, P.1, Q.10, Hồ Chí Minh","['ic ic-bed', 'ic ic-bath', 'ic ic-acreage', '...","['2', '1', '4.3m²', 'Đ.Bắc']"
3,3,https://propzy.vn/mua/nha/hcm/quan-4/id345253,"Bán nhà hẻm quận 4 - Nhà nhỏ, thích hợp gia đì...",id345253,930 triệu,"91,17 triệu/m²","Đ.Tôn Đản, P.10, Q.4, Hồ Chí Minh","['ic ic-bed', 'ic ic-bath', 'ic ic-acreage', '...","['1', '1', '10.2m²', 'Đ.Nam']"
4,4,https://propzy.vn/mua/dat-nen/hcm/huyen-hoc-mo...,"Bán lô đất nông nghiệp, diện tích 96.1m2 Tân H...",id377209,950 triệu,"9,88 triệu/m²","Đ.TL15, X.Tân Hiệp, H.Hóc Môn, Hồ Chí Minh","['ic ic-acreage', 'ic ic-compass']","['96.1m²', 'T.Nam']"


In [9]:
page_19 = list(df.link)
len(page_19)

640

In [ ]:
# df 1 : 640 item 
df1 = get_all(page_19)
df1.head()

In [ ]:
df1.to_excel('propzy_640.xlsx')

In [ ]:
df1.to_csv('propzy_640.csv')

In [10]:
# read page_19_high ( 640 items )
df2 = pd.read_csv('page_19_high.csv')
df2.head()

,Unnamed: 0,link,brief,id,price,price_per_unit,address,facilities_key,facilities_value
0,0,https://propzy.vn/mua/dat-nen/hcm/quan-2/id305298,"Bán lô đất rộng gần 2.000m2, mặt tiền đường Tr...",id305298,610 tỷ,"305,41 triệu/m²","Đ.Trần Não, P.Bình An, Q.2, Hồ Chí Minh","['ic ic-acreage', 'ic ic-compass']","['1997.3m²', 'T.Bắc']"
1,1,https://propzy.vn/mua/nha/hcm/quan-6/id348302,Xường bán Hậu Giang P11 Q6 - 37.4 x 56.59m - V...,id348302,320 tỷ,100 triệu/m²,"Đ.Hậu Giang, P.11, Q.6, Hồ Chí Minh","['ic ic-bed', 'ic ic-bath', 'ic ic-acreage', '...","['1', '2', '3199.7m²', 'Đông']"
2,2,https://propzy.vn/mua/nha/hcm/quan-binh-thanh/...,Bán nhà và xưởng đang kinh doanh ở Bình Quới Q...,id412503,306 tỷ,"111,89 triệu/m²","Đ.Bình Quới, P.28, Q.Bình Thạnh, Hồ Chí Minh","['ic ic-bed', 'ic ic-bath', 'ic ic-acreage', '...","['7', '7', '2734.6m²', 'T.Bắc']"
3,3,https://propzy.vn/mua/dat-nen/hcm/quan-2/id236736,"Bán đất sản xuất nông nghiệp, gần cảng Cát Lái...",id236736,270 tỷ,27 triệu/m²,"Đ.B, P.Thạnh Mỹ Lợi, Q.2, Hồ Chí Minh","['ic ic-acreage', 'ic ic-compass']","['10000m²', 'Đ.Bắc']"
4,4,https://propzy.vn/mua/nha/hcm/quan-go-vap/id40...,Bán tòa nhà mặt tiền đường Quận Gò Vấp - Nhà...,id403577,260 tỷ,"175,71 triệu/m²","Đ.Trần Bình Trọng, P.1, Q.Gò Vấp, Hồ Chí Minh","['ic ic-bed', 'ic ic-bath', 'ic ic-acreage', '...","['81', '81', '1479.7m²', 'Đ.Bắc']"


In [11]:
page_19_high = list(df2.link)

In [14]:
df2 = get_all(page_19_high)
df2.head()

,item_code,item_name,bedroom,bathroom,area,direction,address,details_property,total_price,unit_price,gross_unit_price,payment,utility
0,305298,"Bán lô đất rộng gần 2.000m2, mặt tiền đường Tr...",Na,Na,1997.3 m²,T.Bắc,"Đ.Trần Não, P.Bình An, Q.2, Hồ Chí Minh",[CHI TIẾT BẤT ĐỘNG SẢN\nChiều dài\n60.99 m\nCh...,610 tỷ,Na,"305,4 triệu/m²","6,29 tỷ/tháng",Na
1,348302,Xường bán Hậu Giang P11 Q6 - 37.4 x 56.59m - V...,1,2,3199.7 m²,Đông,"Đ.Hậu Giang, P.11, Q.6, Hồ Chí Minh",[CHI TIẾT BẤT ĐỘNG SẢN\nPhòng ngủ\n1\nPhòng tắ...,320 tỷ,"94,8 triệu/m²",100 triệu/m²,"3,3 tỷ/tháng",Na
2,412503,Bán nhà và xưởng đang kinh doanh ở Bình Quới Q...,7,7,2734.6 m²,T.Bắc,"Đ.Bình Quới, P.28, Q.Bình Thạnh, Hồ Chí Minh",[CHI TIẾT BẤT ĐỘNG SẢN\nPhòng ngủ\n7\nPhòng tắ...,306 tỷ,"111,8 triệu/m²","111,8 triệu/m²","3,15 tỷ/tháng",Di chuyển thuận tiện ra trung tâm\nKhu vực an ...
3,236736,"Bán đất sản xuất nông nghiệp, gần cảng Cát Lái...",Na,Na,10000 m²,Đ.Bắc,"Đ.B, P.Thạnh Mỹ Lợi, Q.2, Hồ Chí Minh",[CHI TIẾT BẤT ĐỘNG SẢN\nChiều dài\n200 m\nChiề...,270 tỷ,Na,27 triệu/m²,"2,78 tỷ/tháng","Gần chợ, siêu thị\nGần trường học\nGần mặt tiề..."
4,403577,Bán tòa nhà mặt tiền đường Quận Gò Vấp - Nhà...,81,81,1479.7 m²,Đ.Bắc,"Đ.Trần Bình Trọng, P.1, Q.Gò Vấp, Hồ Chí Minh",[CHI TIẾT BẤT ĐỘNG SẢN\nPhòng ngủ\n81\nPhòng t...,260 tỷ,"160,6 triệu/m²","175,7 triệu/m²","2,68 tỷ/tháng",Na


In [37]:
df2.to_excel('propzy_640_high.xlsx')

In [12]:
# just to fullfill price range  3-> 24 ty
pages = ['https://propzy.vn/mua/bat-dong-san/hcm/gia-tu-3ty-den-4ty?tags=hcm&type=mua&priceFrom=3ty&priceTo=4ty',
        'https://propzy.vn/mua/bat-dong-san/hcm/gia-tu-5ty-den-7ty?tags=hcm&type=mua&priceFrom=5ty&priceTo=7ty',
        'https://propzy.vn/mua/nha/hcm/gia-tu-7ty-den-8ty?propertyTypeIds=%2211%22&tags=hcm&type=mua&priceFrom=7ty&priceTo=8ty',
        'https://propzy.vn/mua/bat-dong-san/hcm/gia-tu-8ty-den-10ty?tags=hcm&type=mua&priceFrom=8ty&priceTo=10ty',
        'https://propzy.vn/mua/bat-dong-san/hcm/gia-tu-11ty-den-15ty?tags=hcm&type=mua&priceFrom=11ty&priceTo=15ty',
        'https://propzy.vn/mua/bat-dong-san/hcm/gia-tu-16ty-den-24ty?tags=hcm&type=mua&priceFrom=16ty&priceTo=24ty']
def get_all_links():
    kq = []
    for page in pages:
        req = Request(page, headers = {'User-Agent': 'Mozilla/5.0'})
        driver.get(page) 
        link = get_detail_link()
        kq += link
    return kq 

page_medium = get_all_links()
len(page_medium)


192

In [17]:
page_medium

['https://propzy.vn/mua/dat-nen/hcm/quan-thu-duc/id404544',
 'https://propzy.vn/mua/nha/hcm/quan-12/id400709',
 'https://propzy.vn/mua/nha/hcm/quan-phu-nhuan/id370767',
 'https://propzy.vn/mua/nha/hcm/quan-phu-nhuan/id116787',
 'https://propzy.vn/mua/nha/hcm/quan-phu-nhuan/id239218',
 'https://propzy.vn/mua/nha/hcm/quan-phu-nhuan/id360847',
 'https://propzy.vn/mua/nha/hcm/quan-12/id348908',
 'https://propzy.vn/mua/nha/hcm/quan-phu-nhuan/id411041',
 'https://propzy.vn/mua/nha/hcm/quan-binh-thanh/id240042',
 'https://propzy.vn/mua/nha/hcm/quan-9/id401473',
 'https://propzy.vn/mua/nha/hcm/quan-binh-thanh/id325604',
 'https://propzy.vn/mua/nha/hcm/quan-12/id417215',
 'https://propzy.vn/mua/nha/hcm/quan-phu-nhuan/id345201',
 'https://propzy.vn/mua/nha/hcm/quan-binh-tan/id415259',
 'https://propzy.vn/mua/nha/hcm/quan-12/id419547',
 'https://propzy.vn/mua/nha/hcm/quan-12/id416368',
 'https://propzy.vn/mua/nha/hcm/quan-8/id336122',
 'https://propzy.vn/mua/nha/hcm/quan-8/id416780',
 'https://pr

In [9]:
df3 = get_all(page_medium)
df3.head()

,item_code,item_name,bedroom,bathroom,area,direction,address,details_property,total_price,unit_price,gross_unit_price,payment,utility
0,404544,"Bán gấp lô đất thổ cư, 2 mặt tiền hẻm, phường ...",Na,Na,47.3 m²,Đ.Nam,"Đ.Xa Lộ Hà Nội, P.Linh Trung, Q.Thủ Đức, Hồ Ch...",[CHI TIẾT BẤT ĐỘNG SẢN\nChiều dài\n8.98 m\nChi...,"3,1 tỷ",Na,"65,5 triệu/m²",32 triệu/tháng,"Gần chợ, siêu thị\nGần bệnh viện\nGần công viê..."
1,400709,Bán nhà riêng Quận 12 - Gần chợ Hiệp Thành,2,1,52.9 m²,T.Nam,"Đ.Hiệp Thành 13, P.Hiệp Thành, Q.12, Hồ Chí Minh",[CHI TIẾT BẤT ĐỘNG SẢN\nPhòng ngủ\n2\nPhòng tắ...,3 tỷ,"51,6 triệu/m²","56,7 triệu/m²","30,97 triệu/tháng","Gần chợ, siêu thị\nGần trường học\nGần mặt tiề..."
2,370767,BÁN NHÀ MT PHƯỜNG 9 QUẬN PHÚ NHUẬN,2,2,21.1 m²,Nam,"Đ.Bùi Văn Thêm, P.9, Q.Phú Nhuận, Hồ Chí Minh",[CHI TIẾT BẤT ĐỘNG SẢN\nPhòng ngủ\n2\nPhòng tắ...,"3,2 tỷ","144,9 triệu/m²","151,6 triệu/m²","33,04 triệu/tháng","Gần chợ, siêu thị\nGần bệnh viện\nGần công viê..."
3,116787,"Nhà Phú Nhuận hẻm an ninh yên tĩnh, sổ hồng ch...",3,2,19.9 m²,Đ.Nam,"Đ.Phan Đình Phùng, P.1, Q.Phú Nhuận, Hồ Chí Minh",[CHI TIẾT BẤT ĐỘNG SẢN\nPhòng ngủ\n3\nPhòng tắ...,3 tỷ,134 triệu/m²,"150,7 triệu/m²","30,97 triệu/tháng",Na
4,239218,"Chính chủ bán nhà nhỏ, hẻm xe ba gác, nhà cũ t...",2,1,27.52 m²,Không xác định,"Đ.Phan Đình Phùng, P.15, Q.Phú Nhuận, Hồ Chí Minh",[CHI TIẾT BẤT ĐỘNG SẢN\nPhòng ngủ\n2\nPhòng tắ...,"3,4 tỷ","116,5 triệu/m²","123,5 triệu/m²","35,1 triệu/tháng",Na


In [10]:
df3.to_excel('propzy_medium.xlsx')


In [12]:
df1 = pd.read_excel('propzy_640.xlsx') 
df2 = pd.read_excel('propzy_640_high.xlsx')
df3 = pd.read_excel('propzy_medium.xlsx')
df = pd.concat([df1, df2, df3], ignore_index=True )
df.head()
df = df.iloc[:,1:]
df



,item_code,item_name,bedroom,bathroom,area,direction,address,details_property,total_price,unit_price,gross_unit_price,payment,utility
0,387193,Bán Chung Cư Cũ Gò Công Quận 5 - Gần Chợ Kim Biên,1,1,14 m²,Đông,"Đ.Gò Công, P.13, Q.5, Hồ Chí Minh",['CHI TIẾT BẤT ĐỘNG SẢN\nPhòng ngủ\n1\nPhòng t...,750 triệu,Na,"53,5 triệu/m²","7,74 triệu/tháng",Na
1,417224,Bán căn hộ nhà ở xã hội huyện Bình Chánh,1,1,34.91 m²,Không xác định,"Đ.Hương Lộ 80, X.Vĩnh Lộc A, H.Bình Chánh, Hồ ...",['CHI TIẾT BẤT ĐỘNG SẢN\nPhòng ngủ\n1\nPhòng t...,780 triệu,Na,"22,3 triệu/m²","8,05 triệu/tháng",Na
2,391821,Bán nhà hẻm an ninh Hồ Thị Kỷ P1 Q10 - Nhà kiê...,2,1,4.3 m²,Đ.Bắc,"Đ.Hồ Thị Kỷ, P.1, Q.10, Hồ Chí Minh",['CHI TIẾT BẤT ĐỘNG SẢN\nPhòng ngủ\n2\nPhòng t...,900 triệu,"196,5 triệu/m²","209,3 triệu/m²","9,29 triệu/tháng",Na
3,345253,"Bán nhà hẻm quận 4 - Nhà nhỏ, thích hợp gia đì...",1,1,10.2 m²,Đ.Nam,"Đ.Tôn Đản, P.10, Q.4, Hồ Chí Minh",['CHI TIẾT BẤT ĐỘNG SẢN\nPhòng ngủ\n1\nPhòng t...,930 triệu,"83,7 triệu/m²","91,1 triệu/m²","9,6 triệu/tháng","Gần chợ, siêu thị\nGần trường học\nDi chuyển t..."
4,377209,"Bán lô đất nông nghiệp, diện tích 96.1m2 Tân H...",Na,Na,96.1 m²,T.Nam,"Đ.TL15, X.Tân Hiệp, H.Hóc Môn, Hồ Chí Minh",['CHI TIẾT BẤT ĐỘNG SẢN\nChiều dài\n19.4 m\nCh...,950 triệu,Na,"9,8 triệu/m²","9,8 triệu/tháng","Gần chợ, siêu thị\nGần trường học\nGần mặt tiề..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1467,117243,Bán Nhà Mặt Tiền Đường Số Phường Tân Hưng Quận...,4,5,90 m²,Nam,"Đ.Số 4A, P.Tân Hưng, Q.7, Hồ Chí Minh",['CHI TIẾT BẤT ĐỘNG SẢN\nPhòng ngủ\n4\nPhòng t...,"19,5 tỷ","202,4 triệu/m²","216,6 triệu/m²","201,33 triệu/tháng","Gần chợ, siêu thị\nGần công viên, trung tâm\nG..."
1468,345748,Bán nhà riêng Quận Gò Vấp - Nhà được thiết kế ...,Na,5,79.8 m²,Đ.Nam,"Đ.Phạm Huy Thông, P.6, Q.Gò Vấp, Hồ Chí Minh",['CHI TIẾT BẤT ĐỘNG SẢN\nPhòng tắm\n5\nChiều d...,"20,5 tỷ","230,6 triệu/m²","256,8 triệu/m²","211,66 triệu/tháng","Gần chợ, siêu thị\nGần công viên, trung tâm\nG..."
1469,399244,"Bán nhà mặt tiền đường Xô Viết Nghệ Tĩnh, phườ...",3,3,96.5 m²,T.Bắc,"Đ.Xô Viết Nghệ Tĩnh, P.26, Q.Bình Thạnh, Hồ Ch...",['CHI TIẾT BẤT ĐỘNG SẢN\nPhòng ngủ\n3\nPhòng t...,"16,5 tỷ","166,9 triệu/m²","170,9 triệu/m²","170,36 triệu/tháng",Na
1470,357519,BÁN NHÀ 2 MẶT TIỀN ĐƯỜNG TRƯỜNG SA QUẬN 3,5,5,51.3 m²,Nam,"Đ.Trường Sa, P.12, Q.3, Hồ Chí Minh",['CHI TIẾT BẤT ĐỘNG SẢN\nPhòng ngủ\n5\nPhòng t...,16 tỷ,"299,9 triệu/m²","311,8 triệu/m²","165,2 triệu/tháng",Na


In [13]:
df.to_excel('raw_propzy.xlsx')

In [14]:
df.to_csv('raw_propzy.csv')

In [42]:
#  get images 
def get_img(urls):
    res_image_url =[]
    res_image_name = []
    for url in urls: 
        req = Request(url, headers = {'User-Agent': 'Mozilla/5.0'})
        driver.get(url) 
        # name image 
        image_name = url.split('/')[-1][2:]
        res_image_name.append(image_name)
        # get image url 
        image_url = driver.find_element(By.XPATH, '//*[@id="listing-detail-image"]/div[1]/div[1]/img').get_attribute('src')
        res_image_url.append(image_url)

        df = pd.DataFrame(list(zip(res_image_url, res_image_name)), columns = ['image_url', 'image_name'])
    return df 


In [44]:
# get img of houses with low price 
low_df = get_img(page_19)
print(len(low_df))
low_df.to_csv('low_img.csv')
low_df.head()

640


,image_url,image_name
0,https://cdn.propzy.vn/listing/large/2021/07/01...,387193
1,https://cdn.propzy.vn/listing/large/2022/06/16...,417224
2,https://cdn.propzy.vn/listing/large/2021/07/19...,391821
3,https://cdn.propzy.vn/listing/large/2021/03/26...,345253
4,https://cdn.propzy.vn/listing/large/2021/06/05...,377209


In [45]:
low_df

,image_url,image_name
0,https://cdn.propzy.vn/listing/large/2021/07/01...,387193
1,https://cdn.propzy.vn/listing/large/2022/06/16...,417224
2,https://cdn.propzy.vn/listing/large/2021/07/19...,391821
3,https://cdn.propzy.vn/listing/large/2021/03/26...,345253
4,https://cdn.propzy.vn/listing/large/2021/06/05...,377209
...,...,...
635,https://cdn.propzy.vn/listing/large/2021/06/25...,385331
636,https://cdn.propzy.vn/listing/large/2022/06/01...,416715
637,https://cdn.propzy.vn/listing/default_detail.jpg,398140
638,https://cdn.propzy.vn/listing/large/2021/06/30...,387071


In [46]:
low_df.iloc[639,1]

'380404'

In [47]:
import requests
from urllib3.exceptions import InsecureRequestWarning
from urllib3 import disable_warnings
disable_warnings(InsecureRequestWarning)
def save_image(df):
    images = []
    for i in range(len(df)):
        filename = 'images/' + df.iloc[i,1]+'.jpg'
        images.append(df.iloc[i,1]+'.jpg')
        r = requests.get(df.iloc[i,0], allow_redirects = True, verify=False)
        open(filename, 'wb').write(r.content)

In [48]:
save_image(low_df) # 632 ảnh do có 8 item bị trùng 

In [51]:
high_df = get_img(page_19_high)
high_df.to_csv('high_img.csv')
high_df.head()

640


,image_url,image_name
0,https://cdn.propzy.vn/listing/default_detail.jpg,305298
1,https://propzy.vn/propzy_assests/images/defaul...,348302
2,https://cdn.propzy.vn/listing/large/2022/04/06...,412503
3,https://cdn.propzy.vn/listing/default_detail.jpg,236736
4,https://cdn.propzy.vn/listing/large/2021/11/06...,403577


In [52]:
save_image(high_df)

In [53]:
# get img of houses with medium price 
page_medium[:5]

['https://propzy.vn/mua/dat-nen/hcm/quan-thu-duc/id404544',
 'https://propzy.vn/mua/nha/hcm/quan-12/id400709',
 'https://propzy.vn/mua/nha/hcm/quan-phu-nhuan/id370767',
 'https://propzy.vn/mua/nha/hcm/quan-phu-nhuan/id116787',
 'https://propzy.vn/mua/nha/hcm/quan-phu-nhuan/id239218']

In [54]:
medium_df = get_img(page_medium)
print(len(medium_df))
medium_df.head()

192


,image_url,image_name
0,https://cdn.propzy.vn/listing/large/2021/12/08...,404544
1,https://cdn.propzy.vn/listing/large/2021/10/19...,400709
2,https://cdn.propzy.vn/listing/large/2021/05/22...,370767
3,https://cdn.propzy.vn/listing/large/2019/12/21...,116787
4,https://cdn.propzy.vn/listing/large/2020/09/24...,239218


In [56]:
medium_df.to_csv('medium_img.csv')

In [55]:
save_image(medium_df)